In [1]:
import AEMP
import pandas as pd
from pathlib import Path
from urlpath import URL
from sys import exit

BASE_DIR = Path(r'C:\Python\AEMP')
BASE_URL = URL('https://www.pbs.gov.au')
exman_prices_url = BASE_URL / 'info/industry/pricing/ex-manufacturer-price'

new_data = AEMP.download_updates(BASE_DIR, exman_prices_url)
if not isinstance(new_data, pd.DataFrame):
    print('No new data found, exiting.')
    exit()
    
earliest_date = new_data.Date.min()
old_data = AEMP.load_db(BASE_DIR, latest_month_only=True)

APPEND = True

if not isinstance(old_data, pd.DataFrame) or not old_data.Date.max() < earliest_date:
    df = new_data
    APPEND = False
    del old_data
else:
    df = pd.concat([old_data, new_data], join='inner', sort=False)
    
pbs_data = AEMP.PBSData()
text_files_zip = pbs_data.get_latest_PBS_text_files()
atc_map = pbs_data.get_atc_from_text_files(text_files_zip)
item_map = pbs_data.get_item_drug_map()

df = AEMP.perform_lookup(df, item_map, atc_map)

if APPEND:
    df = df.loc[df.Date != df.Date.min()]
    
AEMP.write_db(BASE_DIR, df, append=APPEND)